In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

standings_url = "https://fbref.com/en/comps/26/Super-Lig-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.columns = shooting.columns.droplevel()

C:\Users\oguzo\AppData\Local\Temp\ipykernel_10236\589437873.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\oguzo\AppData\Local\Temp\ipykernel_10236\589437873.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [30]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [32]:
team_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-09,21:00,Süper Lig,Matchweek 1,Fri,Home,W,2.0,1.0,Hatayspor,...,4-1-4-1,NaN,Match Report,NaN,26,9,NaN,NaN,1,1
1,2024-08-16,21:00,Süper Lig,Matchweek 2,Fri,Away,W,2.0,1.0,Konyaspor,...,3-4-1-2,NaN,Match Report,NaN,7,3,NaN,NaN,0,0
2,2024-08-21,21:00,Champions Lg,Play-off round,Wed,Away,L,2.0,3.0,ch Young Boys,...,4-4-2,Daniel Siebert,Match Report,Leg 1 of 2,20,8,NaN,NaN,0,0
3,2024-08-27,22:00,Champions Lg,Play-off round,Tue,Home,L,0.0,1.0,ch Young Boys,...,4-2-3-1,Espen Eskås,Match Report,Leg 2 of 2; Young Boys won,17,4,NaN,NaN,0,0
4,2024-08-31,21:45,Süper Lig,Matchweek 4,Sat,Away,W,5.0,1.0,Adana Demirspor,...,4-2-3-1,Kadir Sağlam,Match Report,NaN,16,7,NaN,NaN,0,1
5,2024-09-14,20:00,Süper Lig,Matchweek 5,Sat,Home,W,5.0,0.0,Rizespor,...,5-4-1,Arda Kardeşler,Match Report,NaN,25,9,NaN,NaN,0,0
6,2024-09-17,20:00,Süper Lig,Matchweek 3,Tue,Home,W,3.0,1.0,Gaziantep FK,...,5-4-1,Cihan Aydın,Match Report,NaN,23,8,NaN,NaN,0,0
7,2024-09-21,20:00,Süper Lig,Matchweek 6,Sat,Away,W,3.0,1.0,Fenerbahçe,...,4-2-3-1,Atilla Karaoğlan,Match Report,NaN,13,6,NaN,NaN,0,0
8,2024-09-25,22:00,Europa Lg,League phase,Wed,Home,W,3.0,1.0,gr PAOK,...,4-2-3-1,Alejandro Hernández,Match Report,NaN,25,9,13.9,0.0,0,0
9,2024-09-28,19:00,Süper Lig,Matchweek 7,Sat,Home,D,3.0,3.0,Kasımpaşa,...,4-1-4-1,Oğuzhan Çakir,Match Report,NaN,7,3,NaN,NaN,0,0


In [34]:
years = list(range(2024, 2020, -1))
all_matches = []

In [36]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

all_matches = []

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        # Mevcut sütunları kontrol et
        available_columns = shooting.columns
        required_columns = ["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]
        columns_to_merge = [col for col in required_columns if col in available_columns]

        try:
            team_data = matches.merge(shooting[["Date"] + columns_to_merge], on="Date")
        except ValueError:
            continue
        
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)


C:\Users\oguzo\AppData\Local\Temp\ipykernel_10236\3940231890.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\oguzo\AppData\Local\Temp\ipykernel_10236\3940231890.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\oguzo\AppData\Local\Temp\ipykernel_10236\3940231890.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\oguzo\AppData\Local\Temp\ipykernel_10236\3940231890.py:28: FutureWarning: Passing literal 

ValueError: No tables found

In [59]:
len(all_matches)

39

In [60]:
match_df = pd.concat(all_matches)

In [61]:
match_df.columns = [c.lower() for c in match_df.columns]

In [62]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,Match Report,NaN,8.0,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,Match Report,NaN,7.0,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,Match Report,NaN,10.0,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,Match Report,NaN,11.0,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United


In [27]:
match_df.to_csv("matches.csv")

NameError: name 'match_df' is not defined

## 15/10

In [10]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import numpy as np

warnings.filterwarnings('ignore')

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim")
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)



try:
    # Web sitesini açma
    browser.get("https://www.sofascore.com/inter-miami-cf-new-york-red-bulls/gabsccKc#id:11911622,tab:statistics")
    
    # Sayfanın tamamen yüklenmesini bekleme
    time.sleep(5)  # Bu adım zorunlu olmayabilir, sayfa yavaş yükleniyorsa artırılabilir

    # Sayfanın sonuna kaydırma
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Performans loglarını alma
    logs_raw = browser.get_log("performance")
    logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

    # "shotmap" kelimesi geçen path'leri arama
    for x in logs:
        if 'shotmap' in x['params'].get('headers', {}).get(':path', ''):
            print(x['params'].get('headers', {}).get(':path'))
            break

    # İsteğe bağlı olarak, kaydırmanın ardından tekrar bir bekleme süresi eklenebilir
    time.sleep(2)

except TimeoutException as e:
    print("Sayfa yükleme sırasında bir hata oluştu:", str(e))

finally:
    # Tarayıcıyı kapatma
    browser.quit()

WebDriverException: Message: HTTP method not allowed


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json

# Chrome ayarları
chrome_options = Options()
chrome_options.add_argument("--headless")
caps = webdriver.DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

# ChromeDriver başlatma
driver = webdriver.Chrome(desired_capabilities=caps, options=chrome_options)

# Web sayfasına git
driver.get("https://www.sofascore.com/inter-miami-cf-new-york-red-bulls/gabsccKc#id:11911622,tab:statistics")

# Performans loglarını alma
logs_raw = driver.get_log("performance")
logs = [json.loads(log['message'])['message'] for log in logs_raw]

# Loglarda 'shotmap' içeren path'leri arama
for log in logs:
    if 'shotmap' in log.get('params', {}).get('headers', {}).get(':path', ''):
        print(log['params']['headers'][':path'])
        break

# Tarayıcıyı kapat
driver.quit()


TypeError: WebDriver.__init__() got an unexpected keyword argument 'desired_capabilities'